In [1]:
import pandas as pd
import numpy as np
import os
import glob as glob

In [20]:
"""for file in glob.glob(os.getcwd()+'\*.csv'):
    print(file)"""

df_tr = pd.read_csv('reco_assignment_training.csv')[0:40000]
df_tr['date'] = pd.to_datetime(df_tr['Tran_dt'])

df_ts = pd.read_csv('reco_assignment_holdout.csv')
df_ts['date'] = pd.to_datetime(df_ts['Tran_dt'])

In [21]:
df_tr

,Customer_num,Tran_dt,Product_num,Tran_qty,date
0,C_203152,2022-01-02,P_3365,0.135,2022-01-02
1,C_1607154,2022-01-02,P_58,1.000,2022-01-02
2,C_84518,2022-01-02,P_21295,1.000,2022-01-02
3,C_2553711,2022-01-02,P_20576,1.000,2022-01-02
4,C_2376922,2022-01-02,P_1095,2.000,2022-01-02
...,...,...,...,...,...
39995,C_2457315,2022-01-25,P_16795,1.000,2022-01-25
39996,C_1302900,2022-01-25,P_23867,1.000,2022-01-25
39997,C_1682893,2022-01-25,P_32225,1.000,2022-01-25
39998,C_126332,2022-01-25,P_14873,1.000,2022-01-25


In [22]:
df_ts

,Customer_num,Tran_dt,Product_num,Tran_qty,date
0,C_1967485,2022-04-01,P_4321,1.00,2022-04-01
1,C_62367,2022-04-01,P_11092,1.00,2022-04-01
2,C_2459818,2022-04-01,P_12854,1.00,2022-04-01
3,C_851377,2022-04-01,P_9160,1.00,2022-04-01
4,C_2351826,2022-04-01,P_17207,1.00,2022-04-01
...,...,...,...,...,...
351392,C_354954,2022-05-31,P_14033,1.00,2022-05-31
351393,C_295585,2022-05-31,P_4822,0.99,2022-05-31
351394,C_829221,2022-05-31,P_25252,1.00,2022-05-31
351395,C_1904879,2022-05-31,P_2360,1.00,2022-05-31


In [23]:
cust = df_tr['Customer_num'].unique()
prod = df_tr['Product_num'].unique()

cust_ts = df_ts['Customer_num'].unique()
prod_ts = df_ts['Product_num'].unique()

cust_new = [i for i in cust_ts if i not in cust]
prod_new = [i for i in prod_ts if i not in prod]
len(cust_new), len(prod_new)

(2268, 8668)

In [24]:
len(prod)

8531

###### There isnt any new customer in the holdout set although there are new product in the holdout set

In [25]:
df_tr

,Customer_num,Tran_dt,Product_num,Tran_qty,date
0,C_203152,2022-01-02,P_3365,0.135,2022-01-02
1,C_1607154,2022-01-02,P_58,1.000,2022-01-02
2,C_84518,2022-01-02,P_21295,1.000,2022-01-02
3,C_2553711,2022-01-02,P_20576,1.000,2022-01-02
4,C_2376922,2022-01-02,P_1095,2.000,2022-01-02
...,...,...,...,...,...
39995,C_2457315,2022-01-25,P_16795,1.000,2022-01-25
39996,C_1302900,2022-01-25,P_23867,1.000,2022-01-25
39997,C_1682893,2022-01-25,P_32225,1.000,2022-01-25
39998,C_126332,2022-01-25,P_14873,1.000,2022-01-25


In [26]:
def trans_form(df):    
    df_agg = df_tr.groupby(['Customer_num','Product_num'])['Tran_qty'].sum().reset_index()
    df_agg_tot = df_tr.groupby(['Customer_num'])['Tran_qty'].agg(["mean", "sum"]).reset_index()
    df_tr_agg = df_agg.merge(df_agg_tot, on = ['Customer_num'], how='inner')
    df_tr_agg['rating'] = (df_tr_agg['Tran_qty']*10/df_tr_agg['sum'])
    ### Normalizing the rating converting them into percentage
    return df_tr_agg

In [27]:
df_tr_agg = trans_form(df_tr)

In [28]:
df_tr_agg

,Customer_num,Product_num,Tran_qty,mean,sum,rating
0,C_100082,P_12663,1.000,1.000000,2.000,5.000000
1,C_100082,P_17520,1.000,1.000000,2.000,5.000000
2,C_1001004,P_4251,4.000,1.556250,6.225,6.425703
3,C_1001004,P_7966,0.820,1.556250,6.225,1.317269
4,C_1001004,P_8791,0.405,1.556250,6.225,0.650602
...,...,...,...,...,...,...
37655,C_999487,P_2909,2.000,0.955818,10.514,1.902226
37656,C_999487,P_2945,1.000,0.955818,10.514,0.951113
37657,C_999487,P_37186,0.514,0.955818,10.514,0.488872
37658,C_999487,P_44282,1.000,0.955818,10.514,0.951113


In [29]:
df_tr_agg.columns

Index(['Customer_num', 'Product_num', 'Tran_qty', 'mean', 'sum', 'rating'], dtype='object')

In [30]:
df_tr_agg['Product_num'].nunique()

8531

In [31]:
cus_pro_rat = df_tr_agg.pivot_table(index = 'Customer_num', columns='Product_num', values='rating')
cus_pro_rat.head()

#### Finding similar product

#Lets take P_1000 as the initial product for which we want to find the product which resembles like it

SyntaxError: invalid syntax (2970714202.py, line 6)

#### Finding similar product

In [12]:
def sim_product(product, num_rec = 10, df = cus_pro_rat):
    product_rat = df[product]
    sim_prod = df.corrwith(product_rat)
    sim_df = pd.DataFrame(sim_prod, columns=['correlation'])
    sim_df.dropna(inplace=True)
    return sim_df.sort_values('correlation', ascending=False).loc[sim_df['correlation']>0].head(num_rec)

sim_product('P_1000')

NameError: name 'cus_pro_rat' is not defined

In [32]:
from surprise import SVD

from surprise import Reader, Dataset

#reader = Reader()
reader = Reader(rating_scale=(0, 10))

In [33]:
data = Dataset.load_from_df(df_tr_agg[["Customer_num", "Product_num", "rating"]], reader)

In [13]:
from surprise.model_selection import GridSearchCV

param_grid = {
    "n_epochs": [25, 30, 40, 50],
    "lr_all": [0.005, .01, .015, .02, .03],
    "reg_all": [0.6, .08, .1, .15]
}
#gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)
#grid_result = gs.fit(data)

In [14]:
gs.best_score

NameError: name 'gs' is not defined

In [15]:
# summarize results
print("Best: %f using %s" % (gs.best_score['rmse'], gs.best_params))
means = gs.cv_results['mean_test_rmse']
stds = gs.cv_results['std_test_rmse']
params = gs.cv_results['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

NameError: name 'gs' is not defined

In [17]:
import joblib

#joblib.dump(gs, 'rcm_whole.pkl')

gs1 = joblib.load('rcm_whole.pkl')

In [18]:
gs1.best_score

{'rmse': 0.4848925242795291, 'mae': 0.16000200734434286}

In [19]:
param_best = gs1.best_params['rmse']
param_best

{'n_epochs': 50, 'lr_all': 0.03, 'reg_all': 0.1}

In [34]:
trainset = data.build_full_trainset()

In [35]:
svd = SVD(verbose=True, n_epochs=50, lr_all=0.03, reg_all=0.1)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


In [36]:
trainset.__dict__.keys()

dict_keys(['ur', 'ir', 'n_users', 'n_items', 'n_ratings', 'rating_scale', '_raw2inner_id_users', '_raw2inner_id_items', '_global_mean', '_inner2raw_id_users', '_inner2raw_id_items'])

In [37]:
df_tr_agg['Customer_num'].nunique(), df_tr_agg['Product_num'].nunique(), df_tr_agg.shape

(7567, 8531, (37660, 6))

In [38]:
trainset.__dict__['n_items'], trainset.__dict__['n_users'] , trainset.__dict__['n_ratings']

(8531, 7567, 37660)

In [39]:
import gc
gc.collect()

30

In [40]:
testset = trainset.build_anti_testset()

In [41]:
predictions = svd.test(testset)

In [45]:
from surprise import dump

In [46]:
predictions

[Prediction(uid='C_100082', iid='P_4251', r_ui=2.009293680297398, est=4.0199801879865005, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_7966', r_ui=2.009293680297398, est=2.9449570544909607, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_8791', r_ui=2.009293680297398, est=2.7733081249706752, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_8822', r_ui=2.009293680297398, est=3.478250933134745, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_15473', r_ui=2.009293680297398, est=3.37701224068567, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_22246', r_ui=2.009293680297398, est=3.1626017566891504, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_38318', r_ui=2.009293680297398, est=3.2124962339864918, details={'was_impossible': False}),
 Prediction(uid='C_100082', iid='P_8790', r_ui=2.009293680297398, est=3.1236332320535847, details={'was_impossible': F

In [ ]:
dump.dump('ten_perc_prediction_only', predictions = predictions)

In [44]:
dump.dump('ten_perc_prediction', predictions = predictions, algo = svd)

MemoryError: 

In [31]:
cus_pro_rat['P_10']

Customer_num
C_100082    NaN
C_1001004   NaN
C_1001197   NaN
C_1001322   NaN
C_1001329   NaN
             ..
C_998908    NaN
C_999117    NaN
C_999155    NaN
C_999171    NaN
C_999487    NaN
Name: P_10, Length: 9835, dtype: float64

In [32]:
cus_pro_rat.loc[(cus_pro_rat['P_10']!=np.nan)&(cus_pro_rat['P_10'].isnull()==False)]

Product_num,P_10,P_1000,P_10002,P_10007,P_10008,P_10009,P_1001,P_10010,P_10011,P_10012,...,P_9978,P_9979,P_998,P_9980,P_9983,P_9984,P_9988,P_999,P_9991,P_9992
Customer_num,,,,,,,,,,,,,,,,,,,,,
C_2167316,0.036239,0.019026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.015764,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
svd.predict(uid='C_2167316', iid='P_10')

Prediction(uid='C_2167316', iid='P_10', r_ui=None, est=0.0729574898803669, details={'was_impossible': False})

In [34]:
svd.predict(uid='C_2167316', iid='P_1000')

Prediction(uid='C_2167316', iid='P_1000', r_ui=None, est=0, details={'was_impossible': False})

In [ ]:
def generate_recommendation(user_id, model, df_tr_agg):
    

In [35]:
rat_df = cus_pro_rat.reset_index()

In [45]:
rat_df

Product_num,Customer_num,P_10,P_1000,P_10002,P_10007,P_10008,P_10009,P_1001,P_10010,P_10011,...,P_9978,P_9979,P_998,P_9980,P_9983,P_9984,P_9988,P_999,P_9991,P_9992
0,C_100082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_1001004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_1001197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_1001322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_1001329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,C_998908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9831,C_999117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9832,C_999155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9833,C_999171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
rat_df

Product_num,Customer_num,P_10,P_1000,P_10002,P_10007,P_10008,P_10009,P_1001,P_10010,P_10011,...,P_9978,P_9979,P_998,P_9980,P_9983,P_9984,P_9988,P_999,P_9991,P_9992
0,C_100082,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_1001004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_1001197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_1001322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_1001329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,C_998908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9831,C_999117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9832,C_999155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9833,C_999171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
cus_list = rat_df['Customer_num']

In [59]:
rat_df.iloc[0].isna()

Product_num
Customer_num    False
P_10            False
P_1000           True
P_10002          True
P_10007          True
                ...  
P_9984           True
P_9988           True
P_999            True
P_9991           True
P_9992           True
Name: 0, Length: 17412, dtype: bool

In [60]:
df_tr_agg    

,Customer_num,Product_num,Tran_qty,mean,sum,rating
0,C_100082,P_10088,1.000,1.179762,24.775,0.403633
1,C_100082,P_12661,1.000,1.179762,24.775,0.403633
2,C_100082,P_12663,1.000,1.179762,24.775,0.403633
3,C_100082,P_12932,1.000,1.179762,24.775,0.403633
4,C_100082,P_13610,2.000,1.179762,24.775,0.807265
...,...,...,...,...,...,...
304001,C_999487,P_8394,0.115,1.027317,129.442,0.008884
304002,C_999487,P_8568,1.000,1.027317,129.442,0.077255
304003,C_999487,P_8807,1.000,1.027317,129.442,0.077255
304004,C_999487,P_9301,0.785,1.027317,129.442,0.060645


In [62]:
cust_list = df_tr_agg['Customer_num'].unique()
prod_list = df_tr_agg['Product_num'].unique()

cus_df = pd.DataFrame(cust_list, columns=['Customer_num'])
prod_df = pd.DataFrame(prod_list, columns=['Product_num'])
dum_df = cus_df.merge(prod_df, how='cross')

In [65]:
df_tr_agg_full = dum_df.merge(df_tr_agg, on=['Customer_num','Product_num'], how='left')

In [70]:
df_tr_agg_full.tail()

,Customer_num,Product_num,Tran_qty,mean,sum,rating
171237180,C_999487,P_5520,NaN,NaN,NaN,NaN
171237181,C_999487,P_9712,NaN,NaN,NaN,NaN
171237182,C_999487,P_24281,NaN,NaN,NaN,NaN
171237183,C_999487,P_9799,NaN,NaN,NaN,NaN
171237184,C_999487,P_26990,NaN,NaN,NaN,NaN
